In [15]:
import pyvista as pv
import numpy as np
from scipy.sparse import csr_matrix


In [16]:
def make_sphere(radius=1, nsub=4):
    sphere = pv.Icosphere(radius=radius, nsub=nsub)  # More uniform

    points = sphere.points.astype(np.float32)
    faces = sphere.faces.reshape(-1, 4)[:, 1:]

    n_points = len(points)
    edge_set = set()
    rows, cols, dists = [], [], []
    edge_lengths = []

    for tri in faces:
        for i in range(3):
            a, b = tri[i], tri[(i + 1) % 3]
            edge = tuple(sorted((a, b)))
            if edge not in edge_set:
                edge_set.add(edge)
                dist = np.linalg.norm(points[a] - points[b])
                edge_lengths.append(dist)  
                rows.extend([a, b])
                cols.extend([b, a])
                dists.extend([dist, dist])

    graph = csr_matrix((dists, (rows, cols)), shape=(n_points, n_points))
    h = np.mean(edge_lengths)
    
    # for i in range(10):  # Check the first 10 points
    #     neighbors = graph[i].nonzero()[1]
    #     print(f"Point {i}: {len(neighbors)} neighbors -> {neighbors}")

    return graph, faces, points, sphere, h

In [17]:
for i in range(1, 10):
    graph, faces, points, sphere, h = make_sphere(nsub=i)
    print(f"Sphere with nsub={i}:")
    print(f"Number of points: {len(points)}")
    print(f"Number of faces: {len(faces)}")
    print(f"Average edge length: {h:.2f}")
    print()

Sphere with nsub=1:
Number of points: 42
Number of faces: 80
Average edge length: 0.58

Sphere with nsub=2:
Number of points: 162
Number of faces: 320
Average edge length: 0.30

Sphere with nsub=3:
Number of points: 642
Number of faces: 1280
Average edge length: 0.15

Sphere with nsub=4:
Number of points: 2562
Number of faces: 5120
Average edge length: 0.08



Sphere with nsub=5:
Number of points: 10242
Number of faces: 20480
Average edge length: 0.04

Sphere with nsub=6:
Number of points: 40962
Number of faces: 81920
Average edge length: 0.02

Sphere with nsub=7:
Number of points: 163842
Number of faces: 327680
Average edge length: 0.01



KeyboardInterrupt: 